# Super Scanner Software UI Module

This module was written to set up a simple user interface **(UI)** with **ipywidgets**.

In [ ]:
import tkinter as tk
from tkinter import filedialog 
from ipywidgets import *
from IPython.display import display
import phaseretrieval as phr
import fwdimaging as fwd
import phg
import phgbpy

In [ ]:
def select_folder(**kwargs):
    """
    Select a folder path with tkinter
    """
    initialdir = kwargs.pop('initialdir', 
                            os.path.expanduser("~"))
    title = kwargs.pop('title', 
                       'Please choose a directory,then press OK.')
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(select_folder.__name__,
                  list(kwargs.keys())[-1]))     
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    return filedialog.askdirectory(initialdir=initialdir,
                                   parent=root,title = title, mustexist = True)

In [ ]:
def select_file(**kwargs):
    """
    Select a file path with tkinter
    """    
    initialdir = kwargs.pop('initialdir', 
                            os.path.expanduser("~"))
    filetypes = kwargs.pop('filetypes', [('all files', '.*')]) 
    title = kwargs.pop('title', 
                       'Please choose a file,then press OK.')
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(select_file.__name__,
                  list(kwargs.keys())[-1]))       
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    return filedialog.askopenfilename(initialdir=initialdir,
                                   parent=root,title = title, filetypes = filetypes)

### Simulation

In [ ]:
def toggle_button(btn, val):
    """
    Enable a button when val is true and disable it if val is false
    """
    if val is True:
        btn.disabled = False
        btn.button_style = 'success'
    else:
        btn.disabled = True
        btn.button_style = 'danger'

In [ ]:
def get_dir(path,btn,**kwargs):
    filetypes = kwargs.pop('filetypes',[('all files', '.*')])
    file = kwargs.pop('file','PATH')
    toggle_button(btn,False) 
    if file.lower() == "path":
        obj = select_folder()
    elif file.lower() == "file":
        obj = select_file(filetypes = filetypes)
    if isinstance(obj, str):
        path.value = obj
        toggle_button(btn,True)
    else: 
        path.value = ''
        toggle_button(btn,True)        
        btn.button_style = 'warning'

In [ ]:
def sim_form(width=100):
    path = Text(value='', placeholder='*.stl', 
                disabled=True)
    path.layout.width = '50%'
    btn1 = Button(description='Get stl', button_style='warning')
    getpathbox = HBox([path,btn1]) 
    movetype = RadioButtons(options=['Cam', 'Object', 'Spiral'])
    movetypebox = VBox([Label('Movement Type:'), movetype])
    xysteps = IntSlider(min=8, max=16, value=14, 
                        continuous_update=False)
    xysteps.layout.width='30%'
    xystepbox = VBox([Label('X-Y Steps:'), xysteps])
    zsteps = IntSlider(min=1, max=4, value=3, 
                       continuous_update=False)
    zsteps.layout.width='30%'    
    zstepsbox = VBox([Label('Z Steps:'),zsteps])
    zangles = IntRangeSlider(value=[20, 40],min=10,max=80,step=1,
                             continuous_update=False)
    zanglesbox = VBox([Label('Z-Angles:'),zangles])
    scale = IntSlider(min=35, max=100, value=100, 
                      continuous_update=False)
    scale.layout.width='50%'
    scalebox = VBox([Label('Image Scaling Factor:'), scale])
    width = BoundedIntText(value=2560, min=800,max=10000,step=1,
                           description='Width:')
    width.width = '20%'
    height = BoundedIntText(value=1920, min=600,max=10000,step=1,
                            description='Height:')
    height.width = '20%'
    resolution = HBox([Label("Resolution: "),width,height])
    ssproj = Checkbox(value = False)
    projectbox = VBox([Label('Simulated Project:'), ssproj])
    btn2 = Button(description='Begin Process', button_style='success')
    cont1 = Box([getpathbox,movetypebox,xystepbox,zstepsbox,zanglesbox,
                 resolution,scalebox,projectbox,btn2])    
    cont = [cont1]
    accord = Accordion(children=cont, width=width)
    accord.set_title(0, 'Simulated Set Parameters')
    #Button references
    btn1.on_click(lambda btn: get_dir(path,btn,file = "FILE",
                                      filetypes = [('stl object', '.stl')]))
    btn2.on_click(lambda btn: sim_process(path,movetype,xysteps,
                                          zsteps,zangles,[width,height],
                                          scale,ssproj,btn))
    return accord

In [ ]:
def sim_process(path,movetype,xysteps,zsteps,zangles,size,scale,ssproj,btn):
    if (movetype.value.lower() == 'spiral') or  (movetype.value.lower() == 'object'):
        camloc=(2, 0, 9)
    if (movetype.value.lower() == 'cam'):
        camloc=(0, 0, 3)
    if ssproj.value is True:
        gray = True
    else:
        gray = False
    toggle_button(btn,False)
    phgbpy.take_phg_photos(movetype= movetype.value,
                           z0angle=zangles.value[0],mesh = path.value,
                           zeangle=zangles.value[1],
                           xysteps=xysteps.value, zsteps = zsteps.value,
                           scale = scale.value/100, 
                           size = [size[0].value,size[1].value], 
                           ssproj = ssproj.value,gray = gray, camloc0=camloc)
    toggle_button(btn,True)

### Reconstruction

In [ ]:
def rec_form(width=100,form = "SINGLE"):    
    path = Text(value='', disabled=True)
    path.layout.width = '50%'
    btn1 = Button()    
    getpathbox = HBox([path,btn1])
    dist = IntSlider(value=4,min=2,max=5,step=1, 
                       continuous_update=False)
    distbox = VBox([Label("Led Distance"),dist])
    intbox = VBox([IntSlider(value=12,min=1,max=17,step=1,
                             continuous_update=False),IntSlider(value=18,min=1,max=40,step=1,
                             continuous_update=False),IntSlider(value=36,min=1,max=40,step=1,
                             continuous_update=False)])
    for islider in intbox.children:
        islider.layout.visibility = 'hidden' 
    gridtype = RadioButtons(options=['Grid', 'Ring'],description="Led")  
    gridtypebox = VBox([Label('Led Placement:'), 
                        interactive(led_switcher,led = gridtype,intbox = fixed(intbox))]) 
    btn2 = Button(description='Begin Process', button_style='success')
    cont1 = Box([getpathbox,distbox,gridtypebox,intbox,btn2])
    cont = [cont1]
    accord = Accordion(children=cont, width=width)
    accord.set_title(0, 'Reconstruction Parameters')
    if form.lower() == "single":
        path.placeholder = "Image path"
        btn1.description = "Image"
        btn1.button_style='warning'
        btn1.on_click(lambda btn: get_dir(path,btn,file = "FILE",
                                          filetypes = [('jpg files', 
                                                        ('.jpg','.tiff')),
                                                                              ('all files', '.*')]))
        btn2.on_click(lambda btn: rec_s_process(path,dist,gridtype,intbox,btn))
    if form.lower() == "project":
        path.placeholder = "Get Fourier Project"
        btn1.description = "Get project"
        btn1.button_style='warning'
        btn1.on_click(lambda btn: get_dir(path,btn,file = "PATH"))
        btn2.on_click(lambda btn: rec_process(path,dist,gridtype,intbox,btn))               
    return accord

In [ ]:
def led_switcher(led,intbox):
    for islider in intbox.children:
        islider.layout.visibility = 'hidden' 
    if led.lower() == "ring":
        for i,islider in enumerate(intbox.children):
            islider.description = str(i + 1) + ' Step'
            islider.layout.visibility = 'visible'  
    if led.lower() == "grid":
        intbox.children[0].description = 'Grid'
        intbox.children[0].layout.visibility = "visible"         
    return 

In [ ]:
def rec_process(path,dist,gridtype,intbox,btn):
    leds = list()
    for islider in intbox.children:
        leds.append(islider.value)
    toggle_button(btn, False)
    if gridtype.value.lower() == 'ring':
        phr.simulate_fp_proj(path.value, leds, 
                             dleds=dist.value)
    elif gridtype.value.lower() == 'grid':
        phr.simulate_fp_proj(path.value, leds[0], 
                             dleds=dist.value)
    toggle_button(btn, True)   

In [ ]:
def rec_s_process(path,dist,gridtype,intbox,btn):
    leds = list()
    for islider in intbox.children:
        leds.append(islider.value)
    toggle_button(btn, False)
    bw_img_path = phg.conv_bw_PIL(path.value)
    if gridtype.value.lower() == 'ring':
        _, ring_set_path = fwd.simulate_set(leds=leds,
                                            d=dist.value,
                       amplitude=bw_img_path)
        phr.get_photo(leds=leds, d=dist.value,
              inpath=ring_set_path,
              show=True)
    elif gridtype.value.lower() == 'grid':
        _, grid_set_path = fwd.simulate_set(leds=leds[0],
                                            d=dist.value,
                       amplitude=bw_img_path)
        phr.get_photo(leds=leds[0], d=dist.value,
              inpath=grid_set_path,
              show=True)
    toggle_button(btn, True)       

### Photogrammetry

In [ ]:
def phg_form(width=100):
    """
    A simple UI interface form in accordion style
    """    
    #Step 1 parameters
    path = Text(value='', placeholder='Image Set Path', 
                disabled=True)
    path.layout.width = '50%'
    btn1 = Button(description='Get Image Set', button_style='warning')
    getpathbox = HBox([path,btn1]) 
    scale = IntSlider(min=35, max=100, value=100, 
                      continuous_update=False)
    scale.layout.width='50%'
    scalebox = VBox([Label('Image Scaling Factor:'), scale])
    reslevel = IntSlider(min=0, max=4, value=1, 
                         continuous_update=False)
    reslevel.layout.width='20%'
    reslevelbox = VBox([Label('Resolution Level:'), reslevel])
    bw = Checkbox()
    bwbox = VBox([Label('Use Gray-Scale images:'), bw])
    sample = Checkbox(value = False)
    samplebox = VBox([Label('Use Sample Set:'), sample])
    btn2 = Button(description='Begin Process', 
                 button_style='success')
    cont1 = Box([getpathbox, samplebox, 
                         scalebox, reslevelbox, 
                         bwbox, btn2])   
    #Step 2 parameters
    proj_path = Text(value='', placeholder='Project Path', 
                disabled=True)
    rmodel = RadioButtons(options=['Textured', 'Model'], 
                          description='Mesh:')
    btn3 = Button(description="Get 3D Model", 
                 button_style='danger', disabled=True) 
    cont2 = Box(children=[proj_path,rmodel,btn3])
    cont = [cont1,cont2]
    accord = Accordion(children=cont, width=width)
    accord.set_title(0, 'Step 1 - Photogrammetry Parameters')
    accord.set_title(1, 'Get 3D Model')
    btn1.on_click(lambda btn: get_dir(path,btn,file = "PATH"))
    btn2.on_click(lambda btn: phg_process(path,sample,scale,
                                          reslevel,bw,proj_path,btn,btn3))
    btn3.on_click(lambda btn: phg_open(proj_path,rmodel))
    return accord

In [ ]:
def phg_process(phg_path,sample,scale,reslevel,bw,proj_path,btn,btn1):
    toggle_button(btn, False)   
    toggle_button(btn1, False)
    proj_path.value = phg.create_3d_model(inpath=phg_path.value,
                                        sample=sample.value,
                                          scale=scale.value/100, 
                                        bw=bw.value,
                                          reslevel=reslevel.value)
    toggle_button(btn, True)
    toggle_button(btn1, True)       

In [ ]:
def phg_open(projpath,mesh):
    """
    Open a 3d model with meshlab
    """
    #Open 3d Model
    phg.open_3d_model(projpath.value,
                      mesh=mesh.value)